In [2]:
import pandas as pd
path = '../数据预处理/钢铁.csv'
sh02 = pd.read_csv(path)
sh02.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
sh02 = sh02.fillna(0)
sh02 = sh02.set_index('date')
sh02

,code,open,high,low,close,preclose,volume,amount,turn,pctChg,...,sen_per_para,sum_para,word_max,word_mean,word_phrase,word_var,quality_val,pos_num,neg_num,bias
date,,,,,,,,,,,,,,,,,,,,,
2018-1-2,708,11.45,11.84,11.35,11.72,11.43,5377177,6.240550e+07,1.196501,2.537183,...,43.311905,119,153,20.505113,3.601514,33.871250,0.000000,0,0,0.000000
2018-1-3,708,11.65,11.75,11.58,11.67,11.72,4553994,5.312828e+07,1.013331,-0.426623,...,4.571429,7,13,1.754902,0.248366,2.224241,7.104336,25,14,0.282051
2018-1-4,708,11.61,11.98,11.61,11.81,11.67,5806328,6.858123e+07,1.291993,1.199660,...,18.624405,57,74,10.610711,2.515254,14.712223,0.000000,0,0,0.000000
2018-1-5,708,11.75,11.83,11.68,11.72,11.81,3787077,4.448576e+07,0.842680,-0.762067,...,11.553680,46,61,6.364245,0.820411,9.808869,0.000000,0,0,0.000000
2018-1-8,708,11.70,11.93,11.65,11.86,11.72,5537826,6.554397e+07,1.232248,1.194534,...,28.520563,107,112,14.985500,2.757510,23.135593,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-25,603995,27.10,27.10,25.28,25.40,27.10,3578781,9.251166e+07,3.463600,-6.273100,...,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0.000000
2020-12-28,603995,25.50,25.88,24.38,25.16,25.40,2400651,6.003676e+07,2.323400,-0.944900,...,30.702991,78,85,12.560834,1.904083,12.959943,0.000000,0,0,0.000000
2020-12-29,603995,25.10,26.36,24.70,25.50,25.16,2955600,7.595480e+07,2.860500,1.351400,...,6.855769,21,26,3.317308,0.507251,5.423462,0.000000,0,0,0.000000


In [10]:
class GetData:
    def __init__(self, batch_size=20, n=5):
        path = '../数据预处理/钢铁.csv'
        sh02 = pd.read_csv(path)
        self.data = sh02
        self.batch_size = batch_size
        self.n = n
        
    def getData(self):
        self.data.drop(labels=['Unnamed: 0','code'],axis=1,inplace=True)
        self.data = self.data.fillna(0)
        self.data = self.data.set_index('date')
        self.close_min = self.data['close'].min()
        self.close_max = self.data["close"].max()
        self.data = self.data.apply(lambda x: (x - min(x)) / (max(x) - min(x)))
        self.data = self.data.fillna(0)
        return self.data
    
    def process_data(self):
        n = self.n
        self.getData()
#         feature = []
#         label = []
#         for i in range(len(self.data) - n + 2):
#             if i + n < len(self.data):
#                 feature.append(self.data.iloc[i: i + n].values.tolist())
#                 label.append(self.data.close.values[i + n])
        feature = [
            self.data.iloc[i: i + n].values.tolist()
            for i in range(len(self.data) - n + 2)
            if i + n < len(self.data)
        ]
        label = [
            self.data.close.values[i + n]
            for i in range(len(self.data) - n + 2)
            if i + n < len(self.data)
        ]
        num = int(len(feature) * 0.9)
        train_x = feature[:num]
        test_x = feature[num:]
        train_y = label[:num]
        test_y = label[num:]
        return train_x, test_x, train_y, test_y
    
    def get_dataloader(self):
        batch_size = self.batch_size
        train_x, test_x, train_y, test_y = self.process_data()
        x_train = torch.tensor(train_x).float()
        x_test = torch.tensor(test_x).float()
        y_train = torch.tensor(train_y).float()
        y_test = torch.tensor(test_y).float()
        train_data = TensorDataset(x_train, y_train)
        train_dataLoader = DataLoader(train_data, batch_size=batch_size)
        test_data = TensorDataset(x_test, y_test)
        test_dataLoader = DataLoader(test_data, batch_size=batch_size)
        return train_data, test_data

In [19]:
data = pd.read_csv('../数据预处理/钢铁.csv')
data.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
data = data.fillna(0)
data = data.set_index('date')
i = list(data['code'].unique())[0]
data = data[data['code']==i]
data.drop(labels=['code'],axis=1,inplace=True)


In [38]:
class GetData:
    def __init__(self, batch_size=20, n=5):
        """
        简单地读取数据
        """
        path = '../数据预处理/钢铁.csv'
        sh02 = pd.read_csv(path)
        self.data = sh02
        self.batch_size = batch_size
        self.n = n
        self.features = []
        self.labels = []
        
    def lll(self):
        self.data.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
        self.data = self.data.fillna(0)
        self.data = self.data.set_index('date')
        self.indnums = len(list(self.data['code'].unique()))
        for i in list(self.data['code'].unique()):
            data = self.data[self.data['code']==i]
            data.drop(labels=['code'],axis=1,inplace=True)
            self.split(data)
            
    def split(self, data):
        n = self.n
        feature = [
            data.iloc[i: i + n].values.tolist()
            for i in range(len(data) - n + 2)
            if i + n < len(data)
        ]
        label = [
            data.close.values[i + n]
            for i in range(len(data) - n + 2)
            if i + n < len(data)
        ]
        #print(len(label))
        self.features += feature
        self.labels += label
        
    def split_all(self):
        self.lll()
        num = int(self.indnums * 0.9)
        train_x = self.features[:num*725]
        test_x = self.features[num*725:]
        train_y = self.labels[:num*725]
        test_y = self.labels[num*725:]
        return train_x, test_x, train_y, test_y

In [39]:
gd = GetData()
train_x, test_x, train_y, test_y = gd.split_all()

In [41]:
len(train_x)

23200

In [43]:
len(test_x)

1709

In [44]:
len(test_y)

1709

In [45]:
len(train_y)

23200

In [1]:
import pandas as pd
path = '../数据预处理/钢铁.csv'
sh02 = pd.read_csv(path)

In [10]:
codelist = list(sh02['code'].unique())
len(codelist)

36

In [5]:
for i in codelist:
    data = sh02[sh02['code']==i]
    print(len(data))

730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
730
20
730
249


In [9]:
zh = sh02[sh02['code']==603995]
zh

,Unnamed: 0,date,code,open,high,low,close,preclose,volume,amount,...,sen_per_para,sum_para,word_max,word_mean,word_phrase,word_var,quality_val,pos_num,neg_num,bias
24840,24840,2019-12-24,603995,27.02,32.43,27.02,32.43,22.52,1069080,3.454529e+07,...,22.543651,29,43,7.865210,1.203992,8.823594,0.000000,0,0,0.000000
24841,24841,2019-12-25,603995,35.67,35.67,30.50,30.62,32.43,27476793,8.775888e+08,...,2.545455,11,9,1.432526,0.323684,1.068977,7.355304,32,18,0.279999
24842,24842,2019-12-26,603995,29.30,29.88,28.28,28.43,30.62,21015204,6.072797e+08,...,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0.000000
24843,24843,2019-12-27,603995,28.11,28.74,27.82,28.02,28.43,12981176,3.660663e+08,...,4.875000,8,48,2.298137,0.210375,20.308630,6.602448,17,16,0.030303
24844,24844,2019-12-30,603995,27.78,27.80,26.79,26.83,28.02,11173391,3.022666e+08,...,31.244661,74,107,14.873527,2.335516,19.758294,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25084,25084,2020-12-25,603995,27.10,27.10,25.28,25.40,27.10,3578781,9.251166e+07,...,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0.000000
25085,25085,2020-12-28,603995,25.50,25.88,24.38,25.16,25.40,2400651,6.003676e+07,...,30.702991,78,85,12.560834,1.904083,12.959943,0.000000,0,0,0.000000
25086,25086,2020-12-29,603995,25.10,26.36,24.70,25.50,25.16,2955600,7.595480e+07,...,6.855769,21,26,3.317308,0.507251,5.423462,0.000000,0,0,0.000000
25087,25087,2020-12-30,603995,26.18,26.18,24.58,25.41,25.50,2684661,6.757254e+07,...,8.015152,17,33,4.117277,0.429402,9.658062,0.000000,0,0,0.000000
